In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

Cloning into 'Recommender-Systems-Challenge-2022'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 619 (delta 107), reused 139 (delta 52), pack-reused 404
Receiving objects: 100% (619/619), 106.18 MiB | 20.90 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Updating files: 100% (254/254), done.


In [2]:
ICM_type_df = pd.read_csv("Dataset/data_ICM_type.csv")
ICM_type_df

items = ICM_type_df.item_id
features = ICM_type_df.feature_id
data = ICM_type_df.data
ICM_type = sps.csr_matrix((data, (items, features)))
ICM_type = ICM_type.astype(dtype = np.int32)
ICM_type.shape

n_users = 41629
n_itemsFromICM = ICM_type.shape[0]

URM_all_dataframe = pd.read_csv("Dataset/Edited/URM_Binary_Ratings.csv")
URM_all_dataframe.columns = ["UserID", "ItemID", "Data"]
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)), shape=(n_users,n_itemsFromICM))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)

In [3]:
UCM_all = sps.vstack([URM_all, ICM_type.T])
UCM_all = sps.csr_matrix(UCM_all)

In [4]:
URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

UCM_train, UCM_test = split_train_in_two_percentage_global_sample(UCM_all, train_percentage = 0.85)
UCM_train, UCM_validation = split_train_in_two_percentage_global_sample(UCM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(UCM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(UCM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1555 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 947 ( 2.3%) Users that have less than 1 test interactions


In [5]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_RP3beta = RP3betaRecommender(URM_all)
recommender_RP3beta.fit(alpha=0.6949339074999242, beta=0.2853519331891143, topK=59, implicit=True)

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 980.88 column/sec. Elapsed time 28.51 sec


In [6]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(UCM_all)
recommender_SLIMElasticNet.fit(topK=585, l1_ratio=0.009929794168572358, alpha=0.0018687691383859428)

SLIMElasticNetRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SLIMElasticNetRecommender: Processed 2568 ( 9.2%) in 5.00 min. Items per second: 8.55
SLIMElasticNetRecommender: Processed 5389 (19.3%) in 10.00 min. Items per second: 8.98
SLIMElasticNetRecommender: Processed 8164 (29.2%) in 15.00 min. Items per second: 9.07
SLIMElasticNetRecommender: Processed 10960 (39.2%) in 20.00 min. Items per second: 9.13
SLIMElasticNetRecommender: Processed 13682 (48.9%) in 25.00 min. Items per second: 9.12
SLIMElasticNetRecommender: Processed 16404 (58.7%) in 30.00 min. Items per second: 9.11
SLIMElasticNetRecommender: Processed 19185 (68.6%) in 35.01 min. Items per second: 9.13
SLIMElasticNetRecommender: Processed 21968 (78.5%) in 40.01 min. Items per second: 9.15
SLIMElasticNetRecommender: Processed 24654 (88.2%) in 45.01 min. Items per second: 9.13
SLIMElasticNetRecommender: Processed 26427 (94.5%) in 50.01 min. Items per second: 8.81
SLIMElasticNetRecommender: Processed 27968 (10

In [7]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender_EaseR = EASE_R_Recommender(URM_all)
recommender_EaseR.fit(topK=None, normalize_matrix=False, l2_norm=225.7403645195658)

EASE_R_Recommender: URM Detected 3461 (12.4%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 15.70 min


In [8]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of three predictions scores
    R = R1*alpha + R2*beta + R3*(1-alpha-beta)
    
    Class from Dacrema exercise modified by Antonio Ercolani
    The original took as input 2 recommender

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * (1-self.alpha-self.beta)

        return item_weights

In [9]:
recommender = DifferentLossScoresHybridRecommender(UCM_all, recommender_SLIMElasticNet, recommender_EaseR, recommender_RP3beta)
recommender.fit(norm=1, alpha = 0.5, beta = 0.3)

DifferentLossScoresHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.


## Submissions

In [10]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [11]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10, remove_seen_flag = True))

In [12]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions/Submission_SLIM_EN_rp3Beta_EaseR_Stacked.csv', index=False)